In [2]:
from urllib.request import Request, urlopen
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [2]:
# Create reference to main webpage that will be used to seek out various car makes and models
url = 'https://www.carspecs.us/'
response = requests.get(url)

In [3]:
# The beautiful soup result of parsing the main page. Will search for links to pursue
bsMain = BeautifulSoup(response.text,"html.parser")

In [4]:
# Indices 24 to 91 are the car links
MainLinkList = bsMain.findAll('a')
MainLinkList[24]['href']

'/cars/acura'

In [5]:
# This gets the specific make domain for appending to the main link
make = MainLinkList[24]['href']

# Appending the link for car make to the homepage link. This leads to the list of years
NewUrl = url + make[1:]

# Updated URL Request
NewResponse = requests.get(NewUrl)

# Updated Beautiful Soup of years
bsYears = BeautifulSoup(NewResponse.text,'html.parser')



In [54]:
# Find all links on the page of car years
bsYears.findAll('a')

# Find the index of the respective year. Initially I'll only consider 2018 and 2019 cars
yr2019 = [i for i, j in enumerate(bsYears.findAll('a')) if '2019' in j][0]
# yr2018 = [i for i, j in enumerate(bsYears.findAll('a')) if '2018' in j][0]

In [55]:
yr2019

[25]

In [8]:
# Get the year part of link to append to make like
year = bsYears.findAll('a')[yr2019]['href']

# New url with years
NewUrl = url + year[1:]

In [9]:
# Selecting the model
NewResponse = requests.get(NewUrl)

bsModels = BeautifulSoup(NewResponse.text,'html.parser')

MainLinkList = bsModels.findAll('a')

ModelLowIndex = [i for i, j in enumerate(bsModels.findAll('a')) if 'Cars' in j][0]
ModelHighIndex = [i for i, j in enumerate(bsModels.findAll('a')) if 'Privacy Policy' in j][0]

In [10]:
model = MainLinkList[ModelLowIndex + 2]['href']

NewUrl = url + model[1:]

NewUrl

'https://www.carspecs.us/cars/2019/acura/mdx'

In [18]:
# At this stage we are at the specs page and need to start reading data into a dataframe.

NewResponse = requests.get(NewUrl)

# Testing the indices using this manual split
BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})[72].get_text().strip().split('\r\n')[0]


'ABS And Driveline Traction Control\nAirbag Occupancy Sensor\nBack-Up Camera\nCurtain 1st, 2nd And 3rd Row Airbags\nDriver Knee Airbag\nDual Stage Driver And Passenger Front Airbags\nDual Stage Driver And Passenger Seat-Mounted Side Airbags\nForward Collision\nLane Departure Warning\nLane Keeping Assist\nOutboard Front Lap And Shoulder Safety Belts -inc: Rear Center 3...\nRear Child Safety Locks\nSide Impact Beams\nTire Specific Low Tire Pressure Warning\nVehicle Stability Assist Electronic Stability Control (ESC)'

In [21]:
print("test","test")

test test


In [40]:
var = 'test2'
df = pd.DataFrame(columns=["test","test2"],index=range(5))
df[var].loc[2] = "test2 2"
df[var].loc[1] = 'Transmission: 9-Speed Automatic -inc: Sequential SportShift w/paddle shifters, 3-mode integrated dynamics system, Electronic Gear Selector and hill start assist'
df

,test,test2
0,NaN,NaN
1,NaN,Transmission: 9-Speed Automatic -inc: Sequenti...
2,NaN,test2 2
3,NaN,NaN
4,NaN,NaN


## TESTING

In [12]:
variables = []
for i in range(72):
    variables.append(BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})[i].get_text().strip().split('\r\n')[0])

In [14]:
link = 'https://www.carspecs.us/'
response = requests.get(link)
makes = []
print(BeautifulSoup(response.text).findAll('a')[24].get_text())
print(BeautifulSoup(response.text).findAll('a')[91].get_text())
for i in range(24,92):
    makes.append(BeautifulSoup(response.text).findAll('a')[i].get_text())

Acura
Volvo


In [25]:
link = 'https://www.carspecs.us/cars/2019/acura/mdx'
NewResponse = requests.get(link)
BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})[15].get_text().strip().split('\r\n')[1]

'2.84'

In [15]:
url = 'https://www.carspecs.us/'
response = requests.get(url)
make = 'acura'
year = str(2019)

# The beautiful soup result of parsing the main page. Will search for links to pursue
bsMain = BeautifulSoup(response.text,"html.parser")

    
ModelPageUrl = url + "cars/" + year + "/" + make

NewResponse = requests.get(ModelPageUrl)
bsModels = BeautifulSoup(NewResponse.text,'html.parser')
LinkList = bsModels.findAll('a')
    
ModelLowIndex = [x for x, y in enumerate(bsModels.findAll('a')) if 'Cars' in y][0]
ModelHighIndex = [x for x, y in enumerate(bsModels.findAll('a')) if 'Privacy Policy' in y][0]
    
print(LinkList[ModelLowIndex+2].get_text())

MDX


In [32]:
url = 'https://www.carspecs.us/cars/2019/acura/tlx/'
NewResponse = requests.get(url)
data = []

len(BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'}))
    

63

In [18]:
# Testing Alfa Romeo which isn't working
# The problem was that it had a space... :/
    # Create reference to main webpage that will be used to seek out various car makes and models
url = 'https://www.carspecs.us/'
response = requests.get(url)
    
    # The beautiful soup result of parsing the main page. Will search for links to pursue
bsMain = BeautifulSoup(response.text,"html.parser")
make = 'alfa romeo'
year = str(2018)

make = make.replace(" ","-")
    
ModelPageUrl = url + "cars/" + year + "/" + make

NewResponse = requests.get(ModelPageUrl)
bsModels = BeautifulSoup(NewResponse.text,'html.parser')
LinkList = bsModels.findAll('a')

print(LinkList)
ModelLowIndex = [x for x, y in enumerate(bsModels.findAll('a')) if 'Cars' in y][0]
#ModelLowIndex = [x for x, y in enumerate(bsModels.findAll('a')) if 'Cars' in y][0]

[<a class="header-logo logo" href="/"><span>car</span>specs<abbr>.us</abbr></a>, <a href="/">Browse by Make</a>, <a href="/car-finder">Car Finder</a>, <a href="/calculator/0-60">0-60 Calculator</a>, <a href="/account/login">Log In</a>, <a href="/account/manage">My Account</a>, <a href="/">My Favorites</a>, <a href="/account/logoff">Log Out</a>, <a class="js-return-to-search-results" href="/car-finder" rel="nofollow">« Return to search results</a>, <a href="#">Year<span></span></a>, <a href="#">Make<span></span></a>, <a href="#">Price<span></span></a>, <a href="#">Car Type<span></span></a>, <a href="#">Drive Type<span></span></a>, <a href="#">Engine Type<span></span></a>, <a href="#">Transmission<span></span></a>, <a href="#">Performance and MPG<span></span></a>, <a href="#">Doors and Passengers<span></span></a>, <a href="#">Weights and Capacities<span></span></a>, <a href="#">Exterior Dimensions<span></span></a>, <a href="#">Interior Dimensions<span></span></a>, <a href="#">Front Suspe

In [38]:
url = 'https://www.carspecs.us/cars/2018/aston-martin/vanquish-s/48913'
NewResponse = requests.get(url)
make = 'aston martin'
model = 'Vanquish S'
    
year = str(year)

data = []
columns = len(BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'}))-1
print("columns:" , columns)
for i in range(columns):
    print(i)
    data.append(BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})[i].get_text().strip().split('\r\n')[1])
        
data.insert(0,make)
data.insert(1,model)
data.insert(2,year)
    
if columns < 72:
    for columns in range(columns,72):
        data.append("null")

columns: 29
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


IndexError: list index out of range

In [47]:
BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})

[<div class="pure-u-1 pure-u-md-1-2">
 <h4>Transmission</h4>
  8-speed shiftable automatic <br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Drive type</h4>
  Rear wheel drive  <br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>0-60 mph</h4>
 3.70 sec<br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Horsepower</h4>
 580 hp<br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Horsepower RPM</h4>
 7,000 <br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Torque</h4>
 465 ft-lbs.<br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Torque RPM</h4>
 5,500 <br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Cylinders</h4>
 12 <br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Base engine size</h4>
 5.90 L<br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Valves</h4>
 48 <br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Engine type</h4>
 					Gas<br/>
 </div>, <div class="pure-u-1 pure-u-md-1-2">
 <h4>Transmission</h4>
  8-speed shiftabl

In [10]:
# Audi Q8 error
url = 'https://www.carspecs.us/cars/2019/audi/q8'
NewResponse = requests.get(url)
BeautifulSoup(NewResponse.text,'html.parser')#.findAll(True,{'class':'main-car-details'})#.get_text().strip().split('\r\n')[1]

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="en-us" http-equiv="content-language"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="upgrade-insecure-requests" http-equiv="Content-Security-Policy">
<title>2019 Audi Q8 3.0 TFSI Premium Plus </title>
<meta content="2019 Audi Q8 3.0 TFSI Premium Plus  | Car specifications and features" name="description"/>
<link href="https://www.carspecs.us/cars/2019/audi/q8/77722" rel="canonical"/>
<link href="/cdn/photobox.css" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Roboto:300,700,900" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"/>
<link href="/cdn/common?v=ZGlJvCB-WFaorNPp0YBUhOU0lfY4YK269Kku-ihiSLQ1" rel="stylesheet">
<script>
	  (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
	  (i[r]

In [3]:
# This assumes that all cars have the same variables
link = 'https://www.carspecs.us/cars/2019/bmw/i8'
NewResponse = requests.get(link)
variables = []
for i in range(50):
    variables.append(BeautifulSoup(NewResponse.text,
                                   'html.parser')
                                    .findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})[i]
                                    .get_text().strip()
                                    .split('\r\n')[0])
variables.insert(0,'Make')
variables.insert(1,'Model')
variables.insert(2,'Year')
CarData = pd.DataFrame(columns = variables,index=range(2000))

#makes = CarData.columns
CarData.index['Year']

ValueError: invalid literal for int() with base 10: 'Year'

In [26]:
link = 'https://www.carspecs.us/cars/2019/cadillac/xt5'
NewResponse = requests.get(link)
BeautifulSoup(NewResponse.text,'html.parser')
BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'main-car-details'})[0].get_text().strip().split('from ')[1].split('\r\n')[0]

'$50,995'